# NEURAL PREDICTIVE BELIEF REPRESENTATIONS, DeepMind, 2018

## 요약 

* 부분 관측만 가능한 환경(ex. POMDP)에서 비지도학습을 통해서 환경에 대한 좋은 잠재 표현을 구하는 문제
* 좋은 잠재 표현(representation)이란 belief state를 잘 encoding해야 하는 것
* 우리는 딥러닝 모델을 통해서 이러한 belief representation을 구하는 것을 제안
* 좋은 표현을 얻기 위해서 one-step-prediction 기제와 CPC(constrastive predictive coding) 기반 multi-step prediction 기제를 사용
* 결론적으로 action-dependent CPC multi-step prediction이 매우 복잡한 환경에 대한 잠재 표현을 찾는 데에 효과적이었다. 



## 도입 

### 비지도학습 기반 표현학습(representation learning)

* 목표는 가능한 유용한 정보를 인코딩해서 이를 바탕으로 다양한 tasks를 풀 수 있는 토양을 만드는 것
* 표현 학습은 특히 부분 관측 환경(ex. navigation task with first-person view)에서 중요
  * 환경을 부분적으로만 보거나 noisy한 관측을 하는 경우 
* 이러한 환경에서는 에이전트는 해당 환경의 상태에 대한 불확실성을 충분히 반영한 믿음 상태(belief state)를 구축하는 것이 중요
* 이러한 belief state는 미래 상태 예측이나 미래 관측 예측을 가능케 하는 충분 통계(sufficient statistic)이다. 

### 표현학습의 quality 평가 

* black-box approach
  * 배운 상태 표현의 퀄러티를 특정 supervised/RL task의 성능으로 간접적으로 측정
* 우리는 glass-box approach를 제안 
  * 배운 표현으로 직접적으로 ground-truth 환경 상태를 예측하고, 이를 통해 직접적으로 표현 학습의 퀄러티를 측정

### 표현학습의 방법들

* one-step frame prediction
  * 바로 다음 frame의 장면(observation)을 예측 
* constrastive prediction coding(CPC, Ooord, 2018)
  * true next sequence인지 fake next sequence인지를 판가름(GAN 접근법)
* CPC|Action
  * 위에 방법에서 action 정보를 추가로 입력으로 주는 경우 
  



## Partially Observable Markov Decision Process (POMDP, Lovejoy, 1991)

![](./images/beliefref/01.png)

![](./images/beliefref/02.png)


## Noise Contrastive estimation (NCE, Gutmann, 2010)

* positive 데이터 분포에서 온 샘플과 negative 데이터 분포에서 온 샘플을 차별화(discriminate)하는 방향으로 학습하는 방법 
  * ex) GAN에서는 true와 fake dist 구분
  * ex) BERT에서는 true next sentence와 fake next scentence 구분
  * 여기서는 true next sequence of observation과 fake next sequence of observation 구분

* 학습 목표 수립과 optimal solution

![](./images/beliefref/03.png)
![](./images/beliefref/04.png)

## 연관 연구들

* POMDP 환경에서의 compact한 belief representaion만 확보하면 이어지는 RL task에서 더 우수한 성능을 낼 수 있다. 
  * Guestrin 2003, ...
* Predictive State Representation(PSR, Littman 2002)에서는
  * 현재까지의 action들로부터 미래의 observation 시퀀스를 예측을 하는 테스트를 해서 좋은 표현 학습 여부를 가렸다.
  * 우리는 이를 거꾸로 활용해서, 미래 관측 예측을 하게끔 학습해서 좋은 표현 학습을 얻고자 한다. 
* 또다른 연구자들은 지도학습을 통해서 좋은 잠재 표현을 얻고자 했다 
  * 우리는 비지도학습
* 또다른 연구자들은 직접적으로 POMDP의 state transition model을 추정하려고 했다. 
  * 우리는 directly가 아니라 implicitly하게...

## Architecture and Algorittm

* PSR에 착안해서 미래 관측을 예측하는 것, 
* direct하게 미래 관측을 예측하기 보다는 CPC접근으로 true 관측열과 false 관측열을 구분하는 것으로 대처
* CPC|action architecture
  * Blue GRU를 사용해서 이때까지의 관측열과 행동열을 encoding해서 current belief 형성
  * current belief는 action GRU(red)를 initialize 하는 데 사용
  * action-GRU는 true 미래 행동열과 결합하여 true/fake 미래 관측열 분간 task를 풀게 한다. 
![](./images/beliefref/05.png)

* one-step frame prediction (Bengio, 2007)
  * current belief와 current action을 바탕으로 one-step 미래 관측 1개를 예측하는 것 
  
* 학습
  * mini-batch 내에서 uniformly하게 negative example 뽑아도 잘 동작 
  * positive와 negative의 비율은 less significant하다.
    * 아래에서는 positive 1건 보여주고, negative 1건 보여주고, 같은 비중으로...

![](./images/beliefref/06.png)

## 실험

### Toy GridWorld

* small grid를 전진/후진/방향틈의 동작으로 돌아다님, 행동은 랜덤으로 택하되, 한번 한 행동은 1-5회 사이에서 반복
* 현 위치에서 5-length만큼만 관측(POMDP)
* 학습한 표현의 평가를 위해서, 별도의 classifier를 학습하여, 주어진 잠재 표현을 바탕으로 에이전트의 위치와 방향을 예측하게 함

* 아래 그림을 보면 
  * 부분 관측의 한계로 인해 에이전트의 위치와 방향에 대한 예측의 불확실성이 잘 표현되어 있다. 
  * 하지만 벽과 같은 결정적인 증거 관측을 통해서 불확실성이 급격히 감소되고 있음을 알 수 있다. 
  * 간접적으로 belief representation이 잘 되있다고 말할 수 있겠다. 
![](./images/beliefref/07.png)

### Deepmind platform 제공 4가지 환경 

* fixed : single room, object 위치 고정
* room : single room, object randomized
* maze : fixed maze, object randomized
* terreain : natural desert and forest feature and objects

![](./images/beliefref/08.png)

### 알고리즘별 비교

* 더 먼 CPC 미래 관측열 예측이 더 낫다. 
  * CPC 30 > CPC 1
  * CPC|Action 30 > CPC|Action 1
  * 그렇다고 multi-step FP를 하는 것은 너무 많은 computtion cost가 들 것이다. 
* 심플환 환경에서는 FP가 동작을 꽤 하지만, terrain처럼 복잡한 환경에 되면 관측 자체를 예측하는 것이 매우 challengin하게 되므롤 CPC 기반이 FP보다 우수하게 된다. 
* 그냥 CPC보다는 CPC|Action이 더 낫다

![](./images/beliefref/09.png)


## 추가 싷험

### INCREASED OBJECT INTERACTION

* 물체 위치가 고정된 fixed에서는 물체 위치가 랜덤한 room이나 maze에 비해서 물체 위치 예측이 더 잘 되었다. 
* room이나 maze에서는 물체가 랜덤하므로 환경의 belief를 추정하는데 중요 요소가 아니다(less significant)
  * 그러므로 물체 예측을 잘 못하는 것이 오히려 당연
* 만약 물체가 환경에서 중요한 역할을 하는 것으로 문제를 바꿔본다면
  * belief는 좀 더 물체를 encoding하는데 집중할 것임
* 이를 위해서 물체를 만나면 agent가 teleport하게 문제를 바꿔봄 
  * teleport 문제의 경우 물체를 좀 더 중시할 것이고, 이는 물체 위치 예측의 정확도를 향상시킬 것으로 기대

![](./images/beliefref/10.png)
![](./images/beliefref/11.png)

### RICHER UNCERTAINTY OVER POSITION

* Toy GridWorld처럼 에이전트의 위치에 대한 불확실성을 더 명확히 하기 위한 문제 고안
* 두개 복도 문제 
  * 복도를 지나갈 때는 어떤 복도인지가 매우 불확실하나 복도 끝을 나와서 turning을 할 때면 어떤 복도인지 아주 명확해짐

![](./images/beliefref/12.png)